In [26]:
import pandas as pd
import plotly.express as px
Scenarios = ['On Premise', 'Exadata', 'ExaCC', 'ExaCC BYOL', 'OCI DBCS']
Infrastructure = [80000, 100000, 90000, 90000, 40000]
License_support = [340000, 280000, 0, 200000, 0]
Universal_credits=[0,0,89000,25000,45000]
width = 0.9       # the width of the bars: can also be len(x) sequence

Totals=list(map(lambda a,b,c: a+b+c, Infrastructure, License_support,Universal_credits))

data={'Scenarios':Scenarios,'Infra':Infrastructure,'License_support':License_support,'UC':Universal_credits}
df=pd.DataFrame(data)
print(df)
fig=px.bar(df,x="Scenarios",y=["Infra","License_support","UC"],text_auto=True)
fig.show()


    Scenarios   Infra  License_support     UC
0  On Premise   80000           340000      0
1     Exadata  100000           280000      0
2       ExaCC   90000                0  89000
3  ExaCC BYOL   90000           200000  25000
4    OCI DBCS   40000                0  45000


De lehet ennél részletesebben is rajzolni ezzel a módszerrel

In [109]:
import pandas as pd
import plotly.graph_objects as go
Scenarios = ['On Premise', 'Exadata', 'ExaCC', 'ExaCC BYOL', 'OCI DBCS']
Infrastructure = [80000, 100000, 90000, 90000, 40000]
License_support = [340000, 280000, 0, 200000, 0]
Universal_credits=[0,0,89000,25000,45000]
width = 0.9       # the width of the bars: can also be len(x) sequence

Totals=list(map(lambda a,b,c: a+b+c, Infrastructure, License_support,Universal_credits))

data={'Scenarios':Scenarios,'Infra':Infrastructure,'License_support':License_support,'UC':Universal_credits}
df=pd.DataFrame(data)
df.set_index('Scenarios', inplace=True)
fig=go.Figure(
    data=[
        go.Bar(
            name="Infra",
            x=df.index,
            y=df.Infra,
            offsetgroup=0,
            text=list(map(lambda val: 'Infra \n${:,.0f}K'.format(val/1000) if(val>0) else "",df.Infra)),
            textposition="inside"
        ),
         go.Bar(
            name="License_Support",
            x=df.index,
            y=df.License_support,
            offsetgroup=0,
            text=list(map(lambda val: 'Lic. Support \n${:,.0f}K'.format(val/1000) if(val>0) else "",df.License_support)),
            textposition="inside",
            textfont=dict(
                color="white"
            )
        ),       
        go.Bar(
            name="UC",
            x=df.index,
            y=df.UC,
            offsetgroup=0,
            text=list(map(lambda val: 'UC \n${:,.0f}K'.format(val/1000) if(val>0) else "",df.UC)),
            textposition="inside"
        )
    ]
)
fig.update_layout(barmode="stack",bargap=0.1)

totalSum=df.sum(axis="columns")
totalSumText=list(map(lambda val: '{:+.0%} Total \n${:,.0f}K'.format(val/totalSum.max()-1,val/1000) if(val>0) else "",totalSum))
fig.add_trace(go.Scatter(
#    x=df.Scenarios, 
    x=df.index,
    y=totalSum,
    text=totalSumText,
    mode="text",
    textposition='top center',
    textfont=dict(
        size=18,
    ),
    showlegend=False
))
fig.update_yaxes(range=[0,totalSum.max()*1.2])
fig.update_layout(title="4 years cost comparison",width=df.index.size*220)
fig.show()


And now let's try to process some data from the installed base file

In [141]:
import threading
import pandas as pd
import warnings


# Read the available database options into a dataframe
db_options=pd.read_csv("Database Options.csv", sep=',')
# Read the installed base into a dataframe with a ',' separator
df = pd.read_csv("Install Base.csv", sep=',')

#Calculate the license type based on the Product description 
df["License type"] = df.apply(
    lambda row:
        "Processor" if row["Product Description"].find("Processor") > 0
        else "NUP" if row["Product Description"].find("Named") > 0
        else "Unknown", axis=1)

# Sum up the amount of licenses and the ARR which has the same name and license type

licenses=df[["Product Name", "Quantity", "Contract ARR","License type"]][df["License type"] != "Unknown"].groupby(
    ["Product Name","License type"], as_index=False)[["Quantity", "Contract ARR"]].sum()

#Select the licenses which are database options, meaning that the keywords listed in the db_options
#is part of the license name. The DB Option field will also indicate which license it is in the db_option
#dataframe by index of the license entry

licenses["DB_Option"]=licenses.apply(
    lambda row: pd.Series(list(map(
        lambda option,no:
            no if option in row["Product Name"] else -1, db_options["Keyword"], range(db_options.__len__())
        ))).max(),
    axis=1)

#If the license is a DB option, we can extract the list price of CPU and NUP license from the db_option dataframe
#Based on the license type we can fill up the List price column with the proper type of list price.

licenses["List Price CPU"]=licenses.apply(
    lambda row:
        db_options["List price CPU"][row["DB_Option"]] if row["DB_Option"]>0 else 0,
    axis=1)

licenses["List Price NUP"]=licenses.apply(
    lambda row:
        db_options["List Price NUP"][row["DB_Option"]] if row["DB_Option"]>0 
        else 0,
    axis=1)

licenses["License Name"]=licenses.apply(
    lambda row:
        db_options["License name"][row["DB_Option"]] if row["DB_Option"]>0
        else "Not a DB License",
    axis=1)

licenses["List Price"]=licenses.apply(
    lambda row:
        row["List Price CPU"] if row["License type"] == "Processor" else
        row["List Price NUP"] if row["License type"] == "NUP" else
        1,
    axis=1)

#The average discount can be calculated based on the ratio of the ARR/Quantity and the list price of the license type

licenses["Average Discount"]=licenses.apply(
    lambda row:
        (row["Contract ARR"]/row["Quantity"])/row["List Price"]-1 if row["List Price"]>1 else 0,
    axis=1)

# We create a new dataframe called license_printable from licenses, just to do the right ouptut formatting.

licenses_printable=licenses[licenses["DB_Option"]>0]
warnings.filterwarnings('ignore')
licenses_printable["Contract ARR"]=licenses_printable["Contract ARR"].map('${:,.0f}'.format)
licenses_printable["Average Discount"]=licenses_printable["Average Discount"].map('{:+.0%}'.format)
warnings.filterwarnings('default')

print()
print(licenses_printable[["Product Name","License type","Quantity","Contract ARR","Average Discount"]].to_string(index=False))
print('-'*100)
print("Total:{:>51,.0f}{:>13,.0f}{:+17.0%}".format(
    licenses[licenses["DB_Option"]>0]["Quantity"].max(),
    licenses[licenses["DB_Option"]>0]["Contract ARR"].sum(),
    licenses[licenses["DB_Option"]>0].apply(        # this calculates the weighted average. Weight is based on the ARR value
         lambda row:
             row["Contract ARR"]*row["Average Discount"]
          ,axis=1).sum()/licenses["Contract ARR"][licenses["DB_Option"]>0].sum())   
    )



                      Product Name License type  Quantity Contract ARR Average Discount
Database Lifecycle Management Pack    Processor        22      $27,158             -90%
                  Diagnostics Pack    Processor        22      $16,974             -90%
Oracle Database Enterprise Edition    Processor        22     $196,539             -81%
                       Tuning Pack    Processor        22      $11,316             -90%
----------------------------------------------------------------------------------------------------
Total:                                                 22      251,987             -83%
